In [21]:
import json

# Load credentials
with open('client-credentials.json') as file:
    client_credentials = json.load(file)

print('Credentials:', list(client_credentials.keys())) # ['client_id', 'client_secret']
print(client_credentials['client_id']) # Client ID
print(client_credentials['client_secret'])

Credentials: ['client_id', 'client_secret']
109707
f5ae46981c4a1e01026422283db25d2851e81dd8


In [22]:
oauth_params = {
    'client_id': client_credentials['client_id'],
    'scope': 'read_all,profile:read_all,activity:read_all',
    'redirect_uri': 'https://localhost',
    'response_type': 'code'
}

In [23]:
from urllib.parse import urlencode

# Generate link that users can copy/paste in their browser to authorize our app
print('https://www.strava.com/oauth/authorize' + '?' + urlencode(oauth_params))

https://www.strava.com/oauth/authorize?client_id=109707&scope=read_all%2Cprofile%3Aread_all%2Cactivity%3Aread_all&redirect_uri=https%3A%2F%2Flocalhost&response_type=code


In [24]:
from urllib.parse import urlparse, parse_qs

# Extract Authorization Code from URL
authorization_response="https://localhost/?state=&code=d553f4d7a51668ec70dd5d3414bb9380fa181b8d&scope=read,activity:read_all,profile:read_all,read_all"
authorization_code = parse_qs(urlparse(authorization_response).query)['code'][0]

urlparse(authorization_response).query # state=&code=...&scope=...

parse_qs(urlparse(authorization_response).query)

{'code': ['d553f4d7a51668ec70dd5d3414bb9380fa181b8d'],
 'scope': ['read,activity:read_all,profile:read_all,read_all']}

In [25]:
import requests

# Exchange Authorization Code for Access Token
r = requests.post('https://www.strava.com/oauth/token', 
                  data={
                    'client_id': client_credentials['client_id'],
                    'client_secret': client_credentials['client_secret'],
                    'code': authorization_code,
                    'grant_type': 'authorization_code'
})

r.status_code # 200

200

In [26]:
# Token saver
def token_saver(token_obj):
    with open('token.json', 'w') as file:
        json.dump(token_obj, file, indent=4)

token_saver(r.json())

In [27]:
# Token loader
def get_token():
    with open('token.json', 'r') as file:
        return json.load(file)

token = get_token()
token.keys() # 'token_type', 'expires_at', 'expires_in', 'refresh_token', 'access_token', 'athlete'

dict_keys(['token_type', 'expires_at', 'expires_in', 'refresh_token', 'access_token', 'athlete'])

In [28]:
# check token validity
from datetime import datetime, timedelta

print('Expires at:', datetime.fromtimestamp(token['expires_at'])) # date, time
print('Expires in:', timedelta(seconds=token['expires_in'])) # time delta


Expires at: 2023-08-09 20:28:20
Expires in: 3:07:49


In [29]:
# Refresh expired Access Tokens
r = requests.post('https://www.strava.com/oauth/token', data={
    'client_id': client_credentials['client_id'],
    'client_secret': client_credentials['client_secret'],
    'refresh_token': token['refresh_token'],
    'grant_type': 'refresh_token'
})
token_saver(r.json())
token = get_token()

In [30]:
# List activities
r = requests.get('https://www.strava.com/api/v3/athlete/activities', params={
    'access_token': token['access_token']
})
r.status_code # 200

200

In [31]:
# Save activities
with open('activities.json', 'w') as file:
    json.dump(r.json(), file, indent=4)

In [32]:
# Load data into DataFrame
import pandas as pd

activities_df = pd.read_json(r.text)
activities_df[['name', 'type', 'distance', 'elapsed_time', 'max_speed']]

,name,type,distance,elapsed_time,max_speed
0,Afternoon Run,Run,6944.1,2103,4.0
1,Lunch Run,Run,9197.5,2889,5.3
2,Lunch Run,Run,10826.5,3560,5.0
3,Lunch Run,Run,9118.8,3070,5.1
4,Morning Run,Run,21349.0,5927,6.5
5,Lunch Run,Run,3643.2,1293,3.6


# Alternative implementation using OAuth2Session()

In [33]:
from requests_oauthlib import OAuth2Session
from getpass import getpass

In [34]:
# Create a session for initialization
init_session = OAuth2Session(
    client_credentials['client_id'],
    redirect_uri='https://localhost',
    scope='read_all,profile:read_all,activity:read_all'
)

In [35]:
# Get authorization link
user_link, state = init_session.authorization_url('https://www.strava.com/oauth/authorize')
print('Visit link:', user_link)

Visit link: https://www.strava.com/oauth/authorize?response_type=code&client_id=109707&redirect_uri=https%3A%2F%2Flocalhost&scope=read_all%2Cprofile%3Aread_all%2Cactivity%3Aread_all&state=8Cl2r1xoaGP6E4RD5Pf2AMnPtdzvHB


In [36]:
authorization_response = 'https://localhost/?state=ufIhnYkexNdKTVjPvFFTIVvA3kHucr&code=2b992c6b920ab1adb4d7b2028c5f92cc52ab2676&scope=read,activity:read_all,profile:read_all,read_all'

In [37]:
# Get Access Token
token = init_session.fetch_token(
    'https://www.strava.com/oauth/token',
    authorization_response=authorization_response,
    include_client_id=True,
    client_secret=client_credentials['client_secret']
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # Get Access Token                                                                           │
│ ❱ 2 token = init_session.fetch_token(                                                            │
│   3 │   'https://www.strava.com/oauth/token',                                                    │
│   4 │   authorization_response=authorization_response,                                           │
│   5 │   include_client_id=True,                                                                  │
│                                                                                                  │
│ /home/miguel/.pyenv/versions/3.9/envs/epfl_extension_school/lib/python3.9/site-packages/requests │
│ _oauthlib/oauth2_session.py:244 in fetch_token                                                   │
│                                                                                                  │
│   241 │   │   │   raise InsecureTransportError()                                                 │
│   242 │   │                                                                                      │
│   243 │   │   if not code and authorization_response:                                            │
│ ❱ 244 │   │   │   self._client.parse_request_uri_response(                                       │
│   245 │   │   │   │   authorization_response, state=self._state                                  │
│   246 │   │   │   )                                                                              │
│   247 │   │   │   code = self._client.code                                                       │
│                                                                                                  │
│ /home/miguel/.pyenv/versions/3.9/envs/epfl_extension_school/lib/python3.9/site-packages/oauthlib │
│ /oauth2/rfc6749/clients/web_application.py:220 in parse_request_uri_response                     │
│                                                                                                  │
│   217 │   │   │   │   │   raise MismatchingStateError()                                          │
│   218 │   │   │   oauthlib.oauth2.rfc6749.errors.MismatchingStateError                           │
│   219 │   │   """                                                                                │
│ ❱ 220 │   │   response = parse_authorization_code_response(uri, state=state)                     │
│   221 │   │   self.populate_code_attributes(response)                                            │
│   222 │   │   return response                                                                    │
│   223                                                                                            │
│                                                                                                  │
│ /home/miguel/.pyenv/versions/3.9/envs/epfl_extension_school/lib/python3.9/site-packages/oauthlib │
│ /oauth2/rfc6749/parameters.py:278 in parse_authorization_code_response                           │
│                                                                                                  │
│   275 │   params = dict(urlparse.parse_qsl(query))                                               │
│   276 │                                                                                          │
│   277 │   if state and params.get('state', None) != state:                                       │
│ ❱ 278 │   │   raise MismatchingStateError()                                                      │
│   279 │                                                                                          │
│   280 │   if 'error' in params:                            

In [38]:
# Create a session for reaching the API
api_session = OAuth2Session(
    client_credentials['client_id'],
    token=token, # pass Access Token
    
    # Automatically refresh expired token
    auto_refresh_url='https://www.strava.com/oauth/token',
    auto_refresh_kwargs={
        'client_id': client_credentials['client_id'],
        'client_secret': client_credentials['client_secret']
    },
    token_updater=token_saver # automatically saves new tokens
)

In [39]:
# List activities
r = api_session.get('https://www.strava.com/api/v3/athlete/activities')
r.status_code # 200

200

In [40]:
activities_df = pd.read_json(r.text)
activities_df[['name', 'type', 'distance', 'elapsed_time', 'max_speed']]

,name,type,distance,elapsed_time,max_speed
0,Afternoon Run,Run,6944.1,2103,4.0
1,Lunch Run,Run,9197.5,2889,5.3
2,Lunch Run,Run,10826.5,3560,5.0
3,Lunch Run,Run,9118.8,3070,5.1
4,Morning Run,Run,21349.0,5927,6.5
5,Lunch Run,Run,3643.2,1293,3.6


# Using custom library - stravalib

In [47]:
import json

# Load credentials
with open('client-credentials.json') as file:
    client_credentials = json.load(file)

print('Credentials:', list(client_credentials.keys())) # ['client_id', 'client_secret']
print(client_credentials['client_id']) # Client ID
print(client_credentials['client_secret'])

Credentials: ['client_id', 'client_secret']
109707
cbce7f53916cce0df92ba620fa61d4d039e08276


In [48]:
from stravalib import Client

# Create client
client = Client()

# Get Authorization URL
user_link = client.authorization_url(
    client_id=client_credentials['client_id'],
    redirect_uri='https://localhost',
    scope=['read_all', 'profile:read_all', 'activity:read_all']
)
print('Visit link:', user_link)

Visit link: https://www.strava.com/oauth/authorize?client_id=109707&redirect_uri=https%3A%2F%2Flocalhost&approval_prompt=auto&scope=read_all%2Cprofile%3Aread_all%2Cactivity%3Aread_all&response_type=code


In [49]:
authorization_code = 'https://localhost/?state=&code=83fb8a10e2ad22279cf12a29296529c4540a081d&scope=read,activity:read_all,profile:read_all,read_all'

In [50]:
# Get Access Token
token = client.exchange_code_for_token(
    client_id=client_credentials['client_id'],
    client_secret=client_credentials['client_secret'],
    code=authorization_code)
token_saver(token)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # Get Access Token                                                                           │
│ ❱ 2 token = client.exchange_code_for_token(                                                      │
│   3 │   client_id=client_credentials['client_id'],                                               │
│   4 │   client_secret=client_credentials['client_secret'],                                       │
│   5 │   code=authorization_code)                                                                 │
│                                                                                                  │
│ /home/miguel/.pyenv/versions/3.9/envs/epfl_extension_school/lib/python3.9/site-packages/stravali │
│ b/client.py:179 in exchange_code_for_token                                                       │
│                                                                                                  │
│    176 │   │                                                                                     │
│    177 │   │                                                                                     │
│    178 │   │   """                                                                               │
│ ❱  179 │   │   return self.protocol.exchange_code_for_token(                                     │
│    180 │   │   │   client_id=client_id, client_secret=client_secret, code=code                   │
│    181 │   │   )                                                                                 │
│    182                                                                                           │
│                                                                                                  │
│ /home/miguel/.pyenv/versions/3.9/envs/epfl_extension_school/lib/python3.9/site-packages/stravali │
│ b/protocol.py:177 in exchange_code_for_token                                                     │
│                                                                                                  │
│   174 │   │   │   │    and expires_at (number of seconds since Epoch when the provided access    │
│   175 │   │   :rtype: dict                                                                       │
│   176 │   │   """                                                                                │
│ ❱ 177 │   │   response = self._request(                                                          │
│   178 │   │   │   "https://{0}/oauth/token".format(self.server),                                 │
│   179 │   │   │   params={                                                                       │
│   180 │   │   │   │   "client_id": client_id,                                                    │
│                                                                                                  │
│ /home/miguel/.pyenv/versions/3.9/envs/epfl_extension_school/lib/python3.9/site-packages/stravali │
│ b/protocol.py:304 in _request                                                                    │
│                                                                                                  │
│   301 │   │   self.rate_limiter(raw.headers)                                                     │
│   302 │   │                                                                                      │
│   303 │   │   if check_for_errors:                                                               │
│ ❱ 304 │   │   │   self._handle_protocol_error(raw)                                               │
│   305 │   │                                                                                      │
│   306 │   │   # 204 = No content                           

In [7]:
import time

# Refresh token if necessary
if time.time() > token['expires_at']:
    token = client.refresh_access_token(
        client_id=client_credentials['client_id'],
        client_secret=client_credentials['client_secret'],
        refresh_token=token['refresh_token'])
    token_saver(token)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 import time                                                                                 │
│    2                                                                                             │
│    3 # Refresh token if necessary                                                                │
│ ❱  4 if time.time() > token['expires_at']:                                                       │
│    5 │   token = client.refresh_access_token(                                                    │
│    6 │   │   client_id=client_credentials['client_id'],                                          │
│    7 │   │   client_secret=client_credentials['client_secret'],                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'token' is not defined

In [110]:
# Get activities
activities = client.get_activities(limit=5)
activities # <BatchedResultsIterator entity=Activity>

for activity in activities:
    print(activity)
    
a = list(activities)[0] # Get the first activity

print('Activity name:', a.name)
print('Distance:', a.distance)
print('Athlete name:', a.athlete.firstname)
print('Average heart rate:', a.average_heartrate)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2 activities = client.get_activities(limit=5)                                                 │
│    3 activities # <BatchedResultsIterator entity=Activity>                                       │
│    4                                                                                             │
│ ❱  5 for activity in activities:                                                                 │
│    6 │   print(activity)                                                                         │
│    7                                                                                             │
│    8 a = list(activities)[0] # Get the first activity                                            │
│                                                                                                  │
│ /home/miguel/.pyenv/versions/3.9/envs/epfl_extension_school/lib/python3.9/site-packages/stravali │
│ b/client.py:1905 in __next__                                                                     │
│                                                                                                  │
│   1902 │   │   raise StopIteration                                                               │
│   1903 │                                                                                         │
│   1904 │   def __next__(self):                                                                   │
│ ❱ 1905 │   │   return self.next()                                                                │
│   1906 │                                                                                         │
│   1907 │   def next(self):                                                                       │
│   1908 │   │   """ """                                                                           │
│                                                                                                  │
│ /home/miguel/.pyenv/versions/3.9/envs/epfl_extension_school/lib/python3.9/site-packages/stravali │
│ b/client.py:1912 in next                                                                         │
│                                                                                                  │
│   1909 │   │   if self.limit and self._counter >= self.limit:                                    │
│   1910 │   │   │   self._eof()                                                                   │
│   1911 │   │   if not self._buffer:                                                              │
│ ❱ 1912 │   │   │   self._fill_buffer()                                                           │
│   1913 │   │   try:                                                                              │
│   1914 │   │   │   result = self._buffer.popleft()                                               │
│   1915 │   │   except IndexError:                                                                │
│                                                                                                  │
│ /home/miguel/.pyenv/versions/3.9/envs/epfl_extension_school/lib/python3.9/site-packages/stravali │
│ b/client.py:1866 in _fill_buffer                                                                 │
│                                                                                                  │
│   1863 │   │   # If we cannot fetch anymore from the server then we're done here.                │
│   1864 │   │   if self._all_results_fetched:                                                     │
│   1865 │   │   │   self._eof()                                                                   │
│ ❱ 1866 │   │   raw_results = self.result_fetcher(          